# Data Integration

This code is specically desgined to integrate the data in a unique dataframe and be able to use it in a model

## Libraries import

In [14]:
import pandas as pd
import polars as pl

## Global variables

In [15]:
PATH_DENSITY_REPORT       = 'density_report.csv'
PATH_HISTORICAL_INCIDENTS = 'historical_incidents.csv'
PATH_PRODUCT_ATTRIBUTES   = 'product_attributes.csv'
PATH_SUPPLIER_SCORECARD   = 'supplier_scorecard.csv'

EXPORT_TOTAL_DATAFRAME    = 'total_dataframe.csv'

## Execution

### 1. Read files

In [17]:
# Reading csv files
df_density_report = pl.read_csv(PATH_DENSITY_REPORT, separator=";")
df_historical_incidents = pl.read_csv(PATH_HISTORICAL_INCIDENTS, separator=";")
df_product_attributes = pl.read_csv(PATH_PRODUCT_ATTRIBUTES, separator=";")
df_supplier_scorecard = pl.read_csv(PATH_SUPPLIER_SCORECARD, separator=";")

# Adjusting date values
df_density_report = df_density_report.with_columns([
    pl.col("DateOfReport").str.strptime(pl.Date, format="%Y-%m-%dT%H:%M:%S.%f")
])

df_historical_incidents = df_historical_incidents.with_columns([
    pl.col("DateOfIncident").str.strptime(pl.Date, format="%Y-%m-%dT%H:%M:%S.%f")
])



/var/folders/fh/z6pvq_495gd075z979mzth4r0000gn/T/ipykernel_7669/1083487986.py:9: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("DateOfReport").str.strptime(pl.Date, format="%Y-%m-%dT%H:%M:%S.%f")
/var/folders/fh/z6pvq_495gd075z979mzth4r0000gn/T/ipykernel_7669/1083487986.py:13: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("DateOfIncident").str.strptime(pl.Date, format="%Y-%m-%dT%H:%M:%S.%f")


### 2. Join files

In [18]:
# Put the same name on the date columns
df_density_report       = df_density_report.rename({"DateOfReport": "Date"})
df_historical_incidents = df_historical_incidents.rename({"DateOfIncident": "Date"})

# Drop repeated columns
df_product_attributes = df_product_attributes.drop(["GarmentType","Material"])

# Join the dataframes
df_joined = df_density_report.join(
    df_historical_incidents,
    on=["Date", "SupplierName", "ProductReference"],
    how="left"
)

df_joined = df_joined.join(
    df_product_attributes,
    on=["ProductReference"],
    how="left"
)

## Export

In [19]:
df_joined.write_csv(EXPORT_TOTAL_DATAFRAME, separator=";")